In [107]:
import pandas as pd
import glob
import os
import numpy as np
import pickle
import statistics
import community
import matplotlib as mlp
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

import networkx as nx

import analysis as an
import pandas as pd
import statsmodels as st
import statsmodels.api as sm
from statsmodels.formula.api import ols

from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
from visbrain.io import download_file, read_stc

from nilearn.connectome import ConnectivityMeasure

from nilearn import plotting
#plt.matshow(rwd_mean_correlation_matrix,cmap='hot')

import numpy as np
import seaborn as sns

from scipy import stats as spy_stats

In [ ]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
summary_dict=an.onetoughjar(os.path.join(basepath,'tmp','summary_dict_11-14-2019_04-33-33'))
# dict_keys(['mean_FC', 'graphs', 'clustering_coeff', 'btn_centrality', 'PC', 'modules'])
# summary_dict['NR']['no']['modules']['partition']
summary_dict['NR']['no']['graphs'].nodes(data=True)

an.grace_graph(summary_dict['NR']['no']['graphs'], 'centrality', 'Normal weight', 1)
an.grace_graph(summary_dict['NR']['ov']['graphs'], 'centrality', 'Overweight', 1)
an.grace_graph(summary_dict['NR']['ob']['graphs'], 'centrality', 'Obese', 1)

# Full graph metrics

In [ ]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
#Load data from pickle if needed
save_dict=an.onetoughjar(os.path.join(basepath,'tmp','save_dict_11-14-2019_02-46-01'))

In [ ]:
stat_dict={'MZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'DZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'NR':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}}}

In [ ]:
stat_dict={'MZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'DZ':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}},
           'NR':{'mean_FC':{'no':{},'ov':{},'ob':{}}, 'clustering_coeff':{'no':{},'ov':{},'ob':{}}, 'btn_centrality':{'no':{},'ov':{},'ob':{}}, 'PC':{'no':{},'ov':{},'ob':{}}}}
for key, value in save_dict.items():
    for subkey,subval in value.items():
        for k,v in subval.items():
            if k in ['mean_FC','clustering_coeff','btn_centrality','PC']:
                stat_dict[key][k][subkey]=pd.DataFrame.from_dict(v, 'index')
                new_name=[]
                for item in list(stat_dict[key][k][subkey].columns[0:100]):
                    new_name.append('IC_%s'%(item+1))
                stat_dict[key][k][subkey].columns=new_name
                stat_dict[key][k][subkey]['group'] = subkey

for key, value in stat_dict.items():
    print(key)
    for k,v in value.items():
        print(k)
        df = pd.concat([v['no'],v['ov']], axis=0)
        df = pd.concat([df,v['ob']], axis=0)
        if k in ['clustering_coeff', 'btn_centrality', 'PC']:
            keys = []
            tables = []
            for variable in list(df.columns)[0:100]:
                m='%s ~ group'%variable
                model = ols(m, data=df).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)

                keys.append(variable)
                tables.append(anova_table)

            df_anova = pd.concat(tables, keys=keys, axis=0)
            raw_p=df_anova['PR(>F)'].dropna()
            fdr=st.stats.multitest.fdrcorrection(raw_p, alpha=0.05, method='indep', is_sorted=False)
            stat_dict[key][k].update({'full':df, 'anova':df_anova, 'FDR':fdr})
        else:
            print(df)
            keys = []
            tables = []
            variable='IC_1'
            m='%s ~ group'%variable
            model = ols(m, data=df).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)

            keys.append(variable)
            tables.append(anova_table)

            df_anova = pd.concat(tables, keys=keys, axis=0)
            raw_p=df_anova['PR(>F)'].dropna()
            fdr=st.stats.multitest.fdrcorrection(raw_p, alpha=0.05, method='indep', is_sorted=False)
            stat_dict[key][k].update({'full':df, 'anova':df_anova, 'FDR':fdr})



In [ ]:
stat_dict['NR'].keys()

In [ ]:
for key, value in stat_dict.items():
    for subkey,subval in value.items():
        for k,v in subval.items():
            if k == 'FDR':
                s = set(v[0])
                if True in s:
                    print('there is a significant value here %s %s'%(key,subkey))
                    print(stat_dict[key][subkey]['FDR'][1])
                    print(stat_dict[key][subkey]['anova'])
                

In [ ]:
df=stat_dict['MZ']['btn_centrality']['full']
m='%s ~ group'%'IC_2'
model = ols(m, data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
model.summary()

Did not find any differences in the nodal metrics between groups in the not related condition  
IC_2 shows significantly different betweenness centrality between the obese and normal weight monozygotic twin subset. Probably due to a false discovery 

In [ ]:
an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',stat_dict,'6_stat_dict')

## Checking functional connectivity 

In [ ]:
def plot_matrices(matrices, matrix_kind):
    n_matrices = len(matrices)
    fig = plt.figure(figsize=(n_matrices * 4, 4))
    for n_subject, matrix in enumerate(matrices):
        plt.subplot(1, n_matrices, n_subject + 1)
        matrix = matrix.copy()  # avoid side effects
        # Set diagonal to zero, for better visualization
        np.fill_diagonal(matrix, 0)
        vmax = np.max(np.abs(matrix))
        title = '{0}, subject {1}'.format(matrix_kind, n_subject)
        plotting.plot_matrix(matrix, vmin=-vmax, vmax=vmax, cmap='RdBu_r',
                             title=title, figure=fig, colorbar=False)
        


In [ ]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
file_dict=an.onetoughjar(os.path.join(basepath,'tmp','file_dict_11-13-2019_06-28-29'))


In [ ]:
file_dict['NR']['ov']['134223']

In [ ]:
listcorr_dict={'MZ':{'no':[],'ov':[],'ob':[]},'DZ':{'no':[],'ov':[],'ob':[]},'NR':{'no':[],'ov':[],'ob':[]}}
for key, value in file_dict.items():
    for subkey, subval in value.items():
        print(subkey)
        for k,v in subval.items():
            listcorr_dict[key][subkey].append(v.to_numpy())

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')

for key, value in listcorr_dict['NR'].items():
    correlation_matrices = correlation_measure.fit_transform(value)
    # All individual coefficients are stacked in a unique 2D matrix.
    print('Correlations of %s subjects are stacked in an array of shape {0}'%key
          .format(correlation_matrices.shape))
    mean_correlation_matrix = correlation_measure.mean_
    print('Mean correlation has shape {0}.'.format(mean_correlation_matrix.shape))


    plt.figure(figsize=(40,25))
    ax = sns.heatmap(mean_correlation_matrix, linewidth=0.5, cmap='coolwarm')

    #Save the figure
    # ax.figure.savefig("/Users/jennygilbert/Documents/betaseries_bevel/reward_coor_mat.png")

    #plot_matrices(rwd_correlation_matrices[:5], 'reward')
    #plot_matrices(rwd_mean_correlation_matrix, 'reward')



    plt.show()

In [ ]:
def corrector(x, alpha):
    results=x[1].ravel()
    mask = np.isfinite(results)
    pval_corrected = np.empty(results.shape)
    pval_corrected.fill(np.nan)
    pval_corrected[mask] = st.stats.multitest.multipletests(results[mask],alpha=alpha,method='fdr_bh')[1]
    p=np.reshape(pval_corrected, (100, 100))
    print(np.nanmin(p))
    ps = 1-p
    print(np.nanmax(ps))
    coor_fig(ps)
    return(p)

In [ ]:
def coor_fig(df):
    plt.figure(figsize=(40,25))
    m=np.nanmin(df)
    M=np.nanmax(df)
    print('The max p value is %f'%M)
    sns.heatmap(df, linewidth=0.5, 
                vmin=m, vmax=1, 
                cmap=sns.cubehelix_palette(10000), 
                cbar_kws={'ticks': [0.0, 0.2, 0.4, 0.5, 0.7, 0.8,0.975 ,1.0]})
    return(plt.show())

In [ ]:
a=listcorr_dict['NR']['no']
b=listcorr_dict['NR']['ov']
c=listcorr_dict['NR']['ob']
test=spy_stats.f_oneway(a, b, c)
anova_results=corrector(test, 0.05)

### Between no and ov & no and ob
p=0.05/2  
p=0.025

In [ ]:
no_ov=spy_stats.ttest_ind(a, b, nan_policy='omit')
no_ov_results=corrector(no_ov, 0.025)

In [ ]:
no_ob=spy_stats.ttest_ind(a, c, nan_policy='omit')
no_ob_results=corrector(no_ob, 0.025)

## Summary
There appears to be no differences in the graph metrics between nodes. Nor a difference in functional connectivity

# Between modules metrics

In [ ]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
#Load data from pickle if needed
summary_dict=an.onetoughjar(os.path.join(basepath,'tmp','summary_dict_11-14-2019_04-33-33'))
for key, value in summary_dict.items():
    for k,v in value.items():
        # community.induced_graph(partition dictionary, graph)
         comm_graph = community.induced_graph(v['modules']['partition'], v['graphs'])
         v.update(comm_graph = comm_graph)

In [ ]:
an.module_fig(summary_dict['NR']['no']['comm_graph'], 'Normal')
an.module_fig(summary_dict['NR']['ov']['comm_graph'], 'Overweight')
an.module_fig(summary_dict['NR']['ob']['comm_graph'], 'Obese')

## Creating subgraphs per module

In [ ]:
unique, counts = np.unique(summary_dict['NR']['no']['modules']['values'], return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
mod_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }

In [ ]:
for key, values in summary_dict.items():
    print(key)
    for k,v in values.items():
        print(k)
        unique, counts = np.unique(summary_dict[key][k]['modules']['values'], return_counts=True)
        for i in unique:
            mod_dict[key][k].update({i:[]})
        for q, w in summary_dict[key][k]['modules']['partition'].items():
            mod_dict[key][k][w].append(q)

In [ ]:
mod_dict['NR']['no'][1]

In [ ]:
summary_dict['NR']['no'].keys()

In [ ]:
subgraph_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }
for key, value in mod_dict.items():
    print(key)
    for k, v in value.items():
        print(k)
        G=summary_dict[key][k]['graphs']
        for q,w in v.items():
            print(q)
            print(w)
#             H=nx.subgraph_view(G,filter_node=w)
            H = G.subgraph(w).copy()
            subgraph_dict[key][k][q]=H
            
    

In [ ]:
subgraph_dict

In [ ]:
an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',subgraph_dict,'7_subgraph_dict')

In [ ]:
subgraph_dict['NR']['no'][0].nodes(data=True)

In [50]:
modstat_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }

In [92]:
for key, value in subgraph_dict.items():
    print(key)
    for subkey,subval in value.items():
        print(subkey)
        liist=list(subgraph_dict[key][subkey].keys())
        for i in liist:
            modstat_dict[key][subkey].update({i:[]})
        for k,v in subval.items():
            print(k)
            print(subkey)
#             print(dict(v.nodes(data=True)))
            modstat_dict[key][subkey][k]=pd.DataFrame.from_dict(dict(subgraph_dict[key][subkey][k].nodes(data=True)), orient='index')
            modstat_dict[key][subkey][k]['group']=subkey


MZ
no
0
no
1
no
2
no
3
no
4
no
5
no
6
no
7
no
ov
0
ov
1
ov
2
ov
3
ov
4
ov
5
ov
6
ov
7
ov
8
ov
ob
0
ob
1
ob
2
ob
3
ob
4
ob
5
ob
6
ob
7
ob
8
ob
DZ
no
0
no
1
no
2
no
3
no
4
no
5
no
ov
0
ov
1
ov
2
ov
3
ov
4
ov
5
ov
6
ov
7
ov
ob
0
ob
1
ob
2
ob
3
ob
4
ob
5
ob
6
ob
7
ob
NR
no
0
no
1
no
2
no
3
no
4
no
5
no
6
no
7
no
ov
0
ov
1
ov
2
ov
3
ov
4
ov
5
ov
6
ov
7
ov
8
ov
9
ov
ob
0
ob
1
ob
2
ob
3
ob
4
ob
5
ob
6
ob
7
ob
8
ob


In [93]:
list(modstat_dict['NR']['ov'].values())

[    centrality  clustering        PC  modules group
 0     0.008991    0.605844 -0.627084        0    ov
 24    0.011428    0.592655  0.443209        0    ov
 31    0.010614    0.607381  0.432075        0    ov
 38    0.005832    0.562821  0.018082        0    ov,
     centrality  clustering          PC  modules group
 1     0.003417    0.729610  -44.001407        1    ov
 5     0.005004    0.557932   -3.313664        1    ov
 8     0.003861    0.711020  -55.644869        1    ov
 9     0.010131    0.485366  -38.915057        1    ov
 11    0.003440    0.703231   -2.473093        1    ov
 21    0.005730    0.659903  -95.888069        1    ov
 28    0.007383    0.507937  -24.241262        1    ov
 35    0.004840    0.636975   -6.041723        1    ov
 36    0.007705    0.483131   -1.434467        1    ov
 41    0.006311    0.678520   -8.984764        1    ov
 43    0.003395    0.655462 -166.050583        1    ov
 59    0.003977    0.575499   -1.268437        1    ov
 72    0.004368    

In [94]:
pandas_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }
for key, item in modstat_dict.items():
    print(key)
    for k, i in item.items():
        print(i)
        pandas_dict[key][k]=pd.concat(list(modstat_dict['NR'][k].values()))
        

MZ
{0:     centrality  clustering        PC  modules group
0     0.008601    0.623027 -0.524893        0    no
24    0.009632    0.609001  0.556957        0    no
31    0.011434    0.596721  0.561884        0    no
38    0.006913    0.535941  0.337075        0    no, 1:     centrality  clustering           PC  modules group
1     0.003276    0.728283  -423.817362        1    no
5     0.004317    0.578629    -6.177036        1    no
6     0.005822    0.627451    -2.640254        1    no
8     0.004574    0.667677 -3959.985355        1    no
9     0.007225    0.514286   -17.489375        1    no
11    0.003974    0.678076    -8.206111        1    no
13    0.004486    0.576102    -3.940682        1    no
14    0.005313    0.652211   -44.522089        1    no
21    0.006557    0.630897 -1274.671038        1    no
23    0.002811    0.677463    -4.325871        1    no
32    0.003887    0.631707    -6.416971        1    no
35    0.003146    0.659091    -3.521804        1    no
39    0.003408

In [95]:
pandas_dict['NR']

{'no':     centrality  clustering           PC  modules group
 0     0.006490    0.643771    -1.032697        0    no
 24    0.012522    0.607639     0.473302        0    no
 31    0.011106    0.610169     0.543334        0    no
 38    0.006565    0.547561     0.164628        0    no
 1     0.002974    0.746860  -136.687992        1    no
 5     0.005470    0.581081    -7.605033        1    no
 6     0.005565    0.633469    -0.958221        1    no
 8     0.003099    0.732367  -212.633022        1    no
 9     0.010938    0.497561   -37.928071        1    no
 11    0.003908    0.691489    -7.163539        1    no
 13    0.005088    0.569231    -1.961011        1    no
 14    0.006568    0.650830   -12.930303        1    no
 21    0.003573    0.707317  -300.492648        1    no
 23    0.002822    0.685897    -2.522887        1    no
 28    0.006788    0.503788    -3.707025        1    no
 32    0.002888    0.673635    -6.466038        1    no
 35    0.003156    0.682353    -5.875814  

In [96]:
an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',pandas_dict,'8_modulemetrics_dict')

already have tmp


In [ ]:
def corrector(x, alpha):
    results=x[1].ravel()
    mask = np.isfinite(results)
    pval_corrected = np.empty(results.shape)
    pval_corrected.fill(np.nan)
    pval_corrected[mask] = st.stats.multitest.multipletests(results[mask],alpha=alpha,method='fdr_bh')[1]
    p=np.reshape(pval_corrected, (100, 100))
    print(np.nanmin(p))
    ps = 1-p
    print(np.nanmax(ps))
    coor_fig(ps)
    return(p)

In [206]:
df=pd.concat(list(pandas_dict['NR'].values()))

In [207]:
df['group'].value_counts()

no    100
ob    100
ov    100
Name: group, dtype: int64

In [208]:
df['group'] = df['group'].astype('category')
df['group'].cat.reorder_categories(['no', 'ov','ob'], inplace=True)
df['modules'] = df['modules'].astype('category')

In [209]:
PCest = ols(formula="PC ~ modules+group+modules*group", data=df).fit()
PCest.summary()

/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     PC   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.030
Date:                Tue, 19 Nov 2019   Prob (F-statistic):              0.428
Time:                        18:22:10   Log-Likelihood:                -3131.0
No. Observations:                 300   AIC:                             6316.
Df Residuals:                     273   BIC:                             6416.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.0371   4322.781   8.59e-06      1.000   -8510.186    8510.260
modules[T.1]              -112.1511   4593.211     -0.024      0.981   -9154.768    8930.465
modules[T.2]             -2031.0807   4655.778     -0.436      0.663   -1.12e+04    7134.710
modules[T.3]               -10.5060   5418.893     -0.002      0.998   -1.07e+04    1.07e+04
modules[T.4]                -2.0796   4804.503     -0.000      1.000   -9460.664    9456.504
modules[T.5]                -9.6475   5195.336     -0.002      0.999   -1.02e+04    1.02e+04
modules[T.6]                -7.4580   6603.157     -0.001      0.999    -1.3e+04     1.3e+04
modules[T.7]              -155.7920   6113.336     -0.025      0.980   -1.22e+04    1.19e+04
modules[T.8]                -9.9335   2525.295     -0.004      0.997   -4981.462    4961.595
modules[T.9]               -27.0307   2709.446     -0.010      0.992   -5361.095    5307.034
group[T.ov]                  0.0294   6113.336   4.81e-06      1.000    -1.2e+04     1.2e+04
group[T.ob]                  0.1146   6113.336   1.87e-05      1.000    -1.2e+04     1.2e+04
modules[T.1]:group[T.ov]    77.1277   6747.866      0.011      0.991   -1.32e+04    1.34e+04
modules[T.2]:group[T.ov]  1706.2893   6593.717      0.259      0.796   -1.13e+04    1.47e+04
modules[T.3]:group[T.ov]     2.1168   7405.802      0.000      1.000   -1.46e+04    1.46e+04
modules[T.4]:group[T.ov]    -0.3704   7531.191  -4.92e-05      1.000   -1.48e+04    1.48e+04
modules[T.5]:group[T.ov]    -0.1825   7347.315  -2.48e-05      1.000   -1.45e+04    1.45e+04
modules[T.6]:group[T.ov]     3.9581   8151.114      0.000      1.000    -1.6e+04    1.61e+04
modules[T.7]:group[T.ov]   150.5202   1.14e+04      0.013      0.990   -2.24e+04    2.27e+04
modules[T.8]:group[T.ov]     7.8900   3964.888      0.002      0.998   -7797.751    7813.531
modules[T.9]:group[T.ov]   -27.0307   2709.446     -0.010      0.992   -5361.095    5307.034
modules[T.1]:group[T.ob]    89.0713   6597.069      0.014      0.989   -1.29e+04    1.31e+04
modules[T.2]:group[T.ob] -7883.5941   6615.152     -1.192      0.234   -2.09e+04    5139.600
modules[T.3]:group[T.ob]     3.7248   7306.834      0.001      1.000   -1.44e+04    1.44e+04
modules[T.4]:group[T.ob]   -12.5795   7149.329     -0.002      0.999   -1.41e+04    1.41e+04
modules[T.5]:group[T.ob]   -31.0188   7507.058     -0.004      0.997   -1.48e+04    1.47e+04
modules[T.6]:group[T.ob]   -11.8697   8542.019     -0.001      0.999   -1.68e+04    1.68e+04
modules[T.7]:group[T.ob]   150.8316   7928.075      0.019      0.985   -1.55e+04    1.58e+04
modules[T.8]:group[T.ob]   -17.8235   4020.603     -0.004      0.996   -7933.152    7897.504
modules[T.9]:group[T.ob]          0          0        nan        nan           0           0
==============

In [210]:
Clusterest = ols(formula="clustering ~ modules+group+modules*group", data=df).fit()
Clusterest.summary()

/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             clustering   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     4.780
Date:                Tue, 19 Nov 2019   Prob (F-statistic):           9.45e-12
Time:                        18:22:10   Log-Likelihood:                 459.84
No. Observations:                 300   AIC:                            -865.7
Df Residuals:                     273   BIC:                            -765.7
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.6023      0.027     21.992      0.000       0.548       0.656
modules[T.1]                 0.0558      0.029      1.919      0.056      -0.001       0.113
modules[T.2]                 0.0480      0.029      1.626      0.105      -0.010       0.106
modules[T.3]                -0.0031      0.034     -0.090      0.929      -0.071       0.065
modules[T.4]                 0.0549      0.030      1.805      0.072      -0.005       0.115
modules[T.5]                 0.1114      0.033      3.385      0.001       0.047       0.176
modules[T.6]                 0.1518      0.042      3.629      0.000       0.069       0.234
modules[T.7]                 0.0792      0.039      2.046      0.042       0.003       0.155
modules[T.8]                 0.0435      0.016      2.717      0.007       0.012       0.075
modules[T.9]                 0.0421      0.017      2.455      0.015       0.008       0.076
group[T.ov]                 -0.0101      0.039     -0.261      0.794      -0.086       0.066
group[T.ob]                 -0.0138      0.039     -0.356      0.722      -0.090       0.062
modules[T.1]:group[T.ov]    -0.0256      0.043     -0.600      0.549      -0.110       0.059
modules[T.2]:group[T.ov]    -0.0054      0.042     -0.130      0.897      -0.088       0.077
modules[T.3]:group[T.ov]     0.0028      0.047      0.059      0.953      -0.090       0.095
modules[T.4]:group[T.ov]    -0.0107      0.048     -0.225      0.822      -0.105       0.083
modules[T.5]:group[T.ov]    -0.0329      0.047     -0.707      0.480      -0.125       0.059
modules[T.6]:group[T.ov]    -0.0486      0.052     -0.941      0.348      -0.150       0.053
modules[T.7]:group[T.ov]     0.1209      0.072      1.669      0.096      -0.022       0.264
modules[T.8]:group[T.ov]     0.0046      0.025      0.183      0.855      -0.045       0.054
modules[T.9]:group[T.ov]     0.0421      0.017      2.455      0.015       0.008       0.076
modules[T.1]:group[T.ob]     0.0368      0.042      0.881      0.379      -0.045       0.119
modules[T.2]:group[T.ob]     0.0114      0.042      0.271      0.786      -0.071       0.094
modules[T.3]:group[T.ob]     0.0118      0.046      0.254      0.799      -0.079       0.103
modules[T.4]:group[T.ob]    -0.0586      0.045     -1.293      0.197      -0.148       0.031
modules[T.5]:group[T.ob]    -0.1059      0.048     -2.226      0.027      -0.199      -0.012
modules[T.6]:group[T.ob]    -0.0662      0.054     -1.224      0.222      -0.173       0.040
modules[T.7]:group[T.ob]    -0.0130      0.050     -0.258      0.797      -0.112       0.086
modules[T.8]:group[T.ob]     0.0389      0.025      1.526      0.128      -0.011       0.089
modules[T.9]:group[T.ob]          0          0        nan        nan           0           0
==============

In [211]:
Clusterest = ols(formula="centrality ~ modules+group+modules*group", data=df).fit()
Clusterest.summary()

/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             centrality   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     7.211
Date:                Tue, 19 Nov 2019   Prob (F-statistic):           3.66e-19
Time:                        18:22:11   Log-Likelihood:                 1470.9
No. Observations:                 300   AIC:                            -2888.
Df Residuals:                     273   BIC:                            -2788.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.0092      0.001      9.739      0.000       0.007       0.011
modules[T.1]                -0.0041      0.001     -4.097      0.000      -0.006      -0.002
modules[T.2]                -0.0049      0.001     -4.874      0.000      -0.007      -0.003
modules[T.3]                -0.0014      0.001     -1.186      0.237      -0.004       0.001
modules[T.4]                -0.0025      0.001     -2.407      0.017      -0.005      -0.000
modules[T.5]                -0.0051      0.001     -4.511      0.000      -0.007      -0.003
modules[T.6]                -0.0065      0.001     -4.498      0.000      -0.009      -0.004
modules[T.7]                -0.0061      0.001     -4.581      0.000      -0.009      -0.003
modules[T.8]                -0.0024      0.001     -4.381      0.000      -0.003      -0.001
modules[T.9]                -0.0027      0.001     -4.615      0.000      -0.004      -0.002
group[T.ov]               4.571e-05      0.001      0.034      0.973      -0.003       0.003
group[T.ob]                 -0.0012      0.001     -0.917      0.360      -0.004       0.001
modules[T.1]:group[T.ov]     0.0002      0.001      0.159      0.874      -0.003       0.003
modules[T.2]:group[T.ov]    -0.0001      0.001     -0.097      0.923      -0.003       0.003
modules[T.3]:group[T.ov]     0.0007      0.002      0.442      0.659      -0.002       0.004
modules[T.4]:group[T.ov]    -0.0027      0.002     -1.634      0.103      -0.006       0.001
modules[T.5]:group[T.ov]     0.0008      0.002      0.503      0.615      -0.002       0.004
modules[T.6]:group[T.ov]     0.0021      0.002      1.187      0.236      -0.001       0.006
modules[T.7]:group[T.ov]    -0.0017      0.002     -0.695      0.487      -0.007       0.003
modules[T.8]:group[T.ov]    -0.0006      0.001     -0.674      0.501      -0.002       0.001
modules[T.9]:group[T.ov]    -0.0027      0.001     -4.615      0.000      -0.004      -0.002
modules[T.1]:group[T.ob]     0.0011      0.001      0.783      0.434      -0.002       0.004
modules[T.2]:group[T.ob]     0.0010      0.001      0.715      0.475      -0.002       0.004
modules[T.3]:group[T.ob]     0.0012      0.002      0.782      0.435      -0.002       0.004
modules[T.4]:group[T.ob]     0.0008      0.002      0.498      0.619      -0.002       0.004
modules[T.5]:group[T.ob]     0.0024      0.002      1.456      0.147      -0.001       0.006
modules[T.6]:group[T.ob]     0.0027      0.002      1.443      0.150      -0.001       0.006
modules[T.7]:group[T.ob]     0.0038      0.002      2.175      0.030       0.000       0.007
modules[T.8]:group[T.ob]    -0.0018      0.001     -2.087      0.038      -0.004      -0.000
modules[T.9]:group[T.ob]          0          0        nan        nan           0           0
==============

In [214]:
df.columns

Index(['centrality', 'clustering', 'PC', 'modules', 'group'], dtype='object')

In [224]:
keys=[]
tables=[]
pands=[]
for variable in list(df.columns)[0:3]:
    print(variable)
    m='%s ~  modules+group+modules*group'%variable
    model = ols(m, data=df).fit()
    keys.append(variable)
    tables.append(model.pvalues)
    n=pd.DataFrame(model.pvalues)
    n.columns = [variable]
    pands.append(n)
x=pd.concat(tables)
mask = np.isfinite(x)
pval_corrected = np.empty(x.shape)
pval_corrected.fill(np.nan)
pval_corrected[mask] = st.stats.multitest.multipletests(x[mask],alpha=0.05,method='fdr_bh')[1]
print(pval_corrected.shape)
# corrp=pd.concat([tables, y], axis=1)
# corrp=pd.concat(tables,pval_corrected)

centrality
clustering
PC
(90,)


/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [246]:
ps1=pd.DataFrame(pval_corrected[0:30])
ps2=pd.DataFrame(pval_corrected[30:60])
ps3=pd.DataFrame(pval_corrected[60:90])

In [248]:
ps1.columns=['centrality_corr']
ps2.columns=['clustering_corr']
ps3.columns=['PC_corr']

In [250]:
total_corr=ps1.merge(ps2, left_index=True, right_index=True)
total_corr=total_corr.merge(ps3, left_index=True, right_index=True)

In [253]:
total_corr.shape

(30, 3)

In [233]:
total=pands[0].merge(pands[1], left_index=True, right_index=True)

In [235]:
total=total.merge(pands[2], left_index=True, right_index=True)

In [252]:
total.shape

(30, 3)

In [227]:
total=pd.concat(pands)

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [259]:
total.reset_index(level=0, inplace=True)

In [260]:
total

,index,centrality,clustering,PC
0,Intercept,1.978048e-19,2.214203e-62,0.999993
1,modules[T.1],5.519477e-05,5.607768e-02,0.980538
2,modules[T.2],1.856237e-06,1.050126e-01,0.663001
3,modules[T.3],2.367273e-01,9.287069e-01,0.998454
4,modules[T.4],1.672805e-02,7.212709e-02,0.999655
5,modules[T.5],9.613025e-06,8.174770e-04,0.998520
6,modules[T.6],1.013652e-05,3.396802e-04,0.999100
7,modules[T.7],7.048537e-06,4.174110e-02,0.979688
8,modules[T.8],1.687691e-05,7.012464e-03,0.996864
9,modules[T.9],6.047866e-06,1.470548e-02,0.992047


In [261]:
alls=total.join(total_corr)

In [262]:
alls

,index,centrality,clustering,PC,centrality_corr,clustering_corr,PC_corr
0,Intercept,1.978048e-19,2.214203e-62,0.999993,8.604510e-18,1.926356e-60,0.999996
1,modules[T.1],5.519477e-05,5.607768e-02,0.980538,4.801945e-04,2.323218e-01,0.999996
2,modules[T.2],1.856237e-06,1.050126e-01,0.663001,5.383087e-05,3.654438e-01,0.999996
3,modules[T.3],2.367273e-01,9.287069e-01,0.998454,6.240994e-01,9.999962e-01,0.999996
4,modules[T.4],1.672805e-02,7.212709e-02,0.999655,9.095877e-02,2.852299e-01,0.999996
5,modules[T.5],9.613025e-06,8.174770e-04,0.998520,1.102346e-04,5.926708e-03,0.999996
6,modules[T.6],1.013652e-05,3.396802e-04,0.999100,1.102346e-04,2.686562e-03,0.999996
7,modules[T.7],7.048537e-06,4.174110e-02,0.979688,1.022038e-04,1.815738e-01,0.999996
8,modules[T.8],1.687691e-05,7.012464e-03,0.996864,1.631434e-04,4.692957e-02,0.999996
9,modules[T.9],6.047866e-06,1.470548e-02,0.992047,1.022038e-04,8.529180e-02,0.999996


In [264]:
alls.round(4)

,index,centrality,clustering,PC,centrality_corr,clustering_corr,PC_corr
0,Intercept,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000
1,modules[T.1],0.0001,0.0561,0.9805,0.0005,0.2323,1.0000
2,modules[T.2],0.0000,0.1050,0.6630,0.0001,0.3654,1.0000
3,modules[T.3],0.2367,0.9287,0.9985,0.6241,1.0000,1.0000
4,modules[T.4],0.0167,0.0721,0.9997,0.0910,0.2852,1.0000
5,modules[T.5],0.0000,0.0008,0.9985,0.0001,0.0059,1.0000
6,modules[T.6],0.0000,0.0003,0.9991,0.0001,0.0027,1.0000
7,modules[T.7],0.0000,0.0417,0.9797,0.0001,0.1816,1.0000
8,modules[T.8],0.0000,0.0070,0.9969,0.0002,0.0469,1.0000
9,modules[T.9],0.0000,0.0147,0.9920,0.0001,0.0853,1.0000


## Summary
There appear to be differences in centrality:
* between module 9 in ov and module 9 in no


In [268]:
subgraph_dict['NR']['no'].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [287]:
edgestat_dict={'MZ':{'no':{}, 'ov':{}, 'ob':{}},
           'DZ':{'no':{}, 'ov':{}, 'ob':{}},
           'NR':{'no':{}, 'ov':{}, 'ob':{}}
         }
for key, value in subgraph_dict['NR'].items():
    print(key)
#     print(value)        
    liist=list(subgraph_dict['NR'][key].keys())
    for i in liist:
        modstat_dict['NR'][key].update({i:[]})
    for k,v in subval.items():
        print(k)
        print(subkey)
#             print(dict(v.nodes(data=True)))
        edgestat_dict['NR'][key][k]=print(nx.to_numpy_matrix(v))



no
0
ob
[[1.         0.53620703 0.46850508 0.369626  ]
 [0.53620703 1.         0.4558616  0.46609469]
 [0.46850508 0.4558616  1.         0.36166069]
 [0.369626   0.46609469 0.36166069 1.        ]]
1
ob
[[ 1.00000000e+00  1.21403872e-01  6.35215000e-02  2.21867500e-01
   4.26272090e-01  1.71431103e-01  4.88541667e-02  3.92768231e-01
   4.36996321e-01  4.81407077e-01  2.32754179e-01  1.69453500e-01
   3.25728141e-01  2.48894590e-01  4.03145679e-01  3.91852564e-01
  -6.24705128e-03  1.24542987e-01  1.04342423e-01  2.14201808e-01]
 [ 1.21403872e-01  1.00000000e+00  8.86766923e-02  2.29812282e-01
   8.63621795e-03  7.85683077e-02  1.36276410e-01  1.12935167e-01
   2.50704756e-01  2.02557269e-01  2.41851449e-01  2.19257167e-01
   2.68959026e-01  1.57264397e-01  3.04780641e-02  1.76313423e-01
   1.93677410e-01  2.34948923e-01  2.30495231e-01  1.08548564e-01]
 [ 6.35215000e-02  8.86766923e-02  1.00000000e+00  1.70947077e-01
   4.25397051e-02  1.08903282e-01  1.73179115e-01  5.78629615e-02
   1

In [288]:
an.adillyofapickle('/Users/gracer/Google Drive/HCP_graph/1200/datasets',edgestat_dict,'9_edgestat_dict')

already have tmp


In [ ]:
def corrector(x, alpha):
    results=x[1].ravel()
    mask = np.isfinite(results)
    pval_corrected = np.empty(results.shape)
    pval_corrected.fill(np.nan)
    pval_corrected[mask] = st.stats.multitest.multipletests(results[mask],alpha=alpha,method='fdr_bh')[1]
    p=np.reshape(pval_corrected, (100, 100))
    print(np.nanmin(p))
    ps = 1-p
    print(np.nanmax(ps))
    coor_fig(ps)
    return(p)

In [ ]:
def coor_fig(df):
    plt.figure(figsize=(40,25))
    m=np.nanmin(df)
    M=np.nanmax(df)
    print('The max p value is %f'%M)
    sns.heatmap(df, linewidth=0.5, 
                vmin=m, vmax=1, 
                cmap=sns.cubehelix_palette(10000), 
                cbar_kws={'ticks': [0.0, 0.2, 0.4, 0.5, 0.7, 0.8,0.975 ,1.0]})
    return(plt.show())

In [ ]:
a=listcorr_dict['NR']['no']
b=listcorr_dict['NR']['ov']
c=listcorr_dict['NR']['ob']
test=spy_stats.f_oneway(a, b, c)
anova_results=corrector(test, 0.05)

### Between no and ov & no and ob
p=0.05/2  
p=0.025

In [ ]:
no_ov=spy_stats.ttest_ind(a, b, nan_policy='omit')
no_ov_results=corrector(no_ov, 0.025)

In [ ]:
no_ob=spy_stats.ttest_ind(a, c, nan_policy='omit')
no_ob_results=corrector(no_ob, 0.025)

# Within module metrics

# Using visbrain to look at the cifti files

In [ ]:
# Scene creation
sc = SceneObj(bgcolor='black', size=(1400, 1000))
# Colorbar default arguments. See `visbrain.objects.ColorbarObj`
CBAR_STATE = dict(cbtxtsz=12, txtsz=10., width=.1, cbtxtsh=3.,
                  rect=(-.3, -2., 1., 4.))
KW = dict(title_size=14., zoom=1.2)


In [ ]:
# Download the annotation file of the left hemisphere lh.aparc.a2009s.annot
path_to_file1 = download_file('lh.aparc.a2009s.annot', astype='example_data')
# Define the brain object (now you should know how to do it)
b_obj_parl = BrainObj('inflated', hemisphere='left', translucent=False)
# Print parcellates included in the file
# print(b_obj_parl.get_parcellates(path_to_file1))
# Finally, parcellize the brain and add the brain to the scene
b_obj_parl.parcellize(path_to_file1)
sc.add_to_subplot(b_obj_parl, row=1, col=1, rotate='left',
                  title='Parcellize using the Desikan Atlas', **KW)


In [ ]:
# Download the lh.sig.nii.gz file
file = download_file('lh.sig.nii.gz', astype='example_data')
# Define the [...] you know
b_obj_fmri = BrainObj('inflated', translucent=False, sulcus=True)
# Add fMRI activation and hide every activation that is under 5.
b_obj_fmri.add_activation(file=file, clim=(5., 20.), hide_under=5,
                          cmap='viridis', hemisphere='left')
sc.add_to_subplot(b_obj_fmri, row=2, col=1, title='Add fMRI activation',
                  rotate='left', **KW)


In [ ]:
# Link the rotation of subplots (row=0, col=1) and (row=1, col=2)
# sc.link((0, 1), (1, 2))
# Screenshot of the scene
# sc.screenshot('ex_brain_obj.png', transparent=True)

sc.preview()